# C Interpreter

# Third Stage

Design a grammar using an a upward translator that accepts the following inputs from the C language:
- `printf` functions from C language.

Add the AST for arithmetic, logic and relation operators.

To develop this we will desing the grammar that accepts `printf` statements and then implement the AST for arithmetic, logic and relation operators.

## 1. Grammar design: Printf statement

We will start developing the *printf* statement:

### C print statement (printf)

In *C* the base structure of the print statement is:

``` 
printf("string [%type]*", [variable_name])
```

Could contains variables or not, and the numbers of tags must match the number of variables.

- printexpr <- `PRINTF` `(` stringexpr printftail
- printftrail <- `,` fact printftail
- printftrail <- `)`  
- stringexpr <- CSTRING
- fact -> `-` fact
- fact -> num
- fact -> `ID`

## Implementing the lexical analyzer (Lexer)

We will reuse the main lexical analyzer implementation, adding the *printf* logic necessary to process the new rules

In [32]:
from sly import Lexer

class Scanner(Lexer):
    tokens = {ID, CNUM, PRINTF, STYPE, CSTRING}
    literals ={'(', ')',',',';'}

    # Ignore whitespace and tabulations

    ignore = ' \t'

    # Regular expressions rules for tokens

    ID = r'[a-zA-Z][\w_]*'
    CSTRING = r'\"(\\.|[^\"])*\"'

    # Special cases
    ID['printf'] = PRINTF
    CSTRING['%'] = STYPE

    @_(r'\d+')
    def CNUM(self, t):
        t.value = int(t.value)
        return t

    # Error handling rule

    def error(self, t):
        print('<-'*10,"Illegal character '{}'".format(t.value[0]), '->'*10)
        self.index += 1
        t.type='Illegal'
        t.value =t.value[0]
        return t

## Testing lexical Analyzer

In [33]:
import pandas as pd 

data = pd.read_csv('../assets/testing/printf_sentences.csv', delimiter="'")
data[['printf_sentences']]

,printf_sentences
0,"printf(""%d %d %d %d"", a, b, c, d);"
1,"printf(""%d %f"", 8, 45);"
2,"printf(""the add is %d the sub is %f"", 8, 45);"


In [34]:
lexer = Scanner()
sentences = data['printf_sentences'].values
pass_or_not = []
all_token_pass = True

for index, sentence in enumerate(sentences):
    print('-' * 80,"{} Lexically Testing sentence: '{}'".format(index, sentence),'-' * 80, sep='\n')
    for token in lexer.tokenize(sentence):
        print(" type = '{}', value = '{}'".format(token.type, token.value))
        if all_token_pass and 'Illegal' in token.type:
            all_token_pass = False
    
    pass_or_not.append('Pass') if all_token_pass else pass_or_not.append('FAIL')
    all_token_pass = True

data['Test'] = pass_or_not

--------------------------------------------------------------------------------
0 Lexically Testing sentence: 'printf("%d %d %d %d", a, b, c, d);'
--------------------------------------------------------------------------------
 type = 'PRINTF', value = 'printf'
 type = '(', value = '('
 type = 'CSTRING', value = '"%d %d %d %d"'
 type = ',', value = ','
 type = 'ID', value = 'a'
 type = ',', value = ','
 type = 'ID', value = 'b'
 type = ',', value = ','
 type = 'ID', value = 'c'
 type = ',', value = ','
 type = 'ID', value = 'd'
 type = ')', value = ')'
 type = ';', value = ';'
--------------------------------------------------------------------------------
1 Lexically Testing sentence: 'printf("%d %f", 8, 45);'
--------------------------------------------------------------------------------
 type = 'PRINTF', value = 'printf'
 type = '(', value = '('
 type = 'CSTRING', value = '"%d %f"'
 type = ',', value = ','
 type = 'CNUM', value = '8'
 type = ',', value = ','
 type = 'CNUM', value